In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/My Drive/Recsys-2019/sequence_classifier

/content/drive/My Drive/Recsys-2019/sequence_classifier


In [0]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [0]:
import _pickle as cPickle

In [0]:
data_path = 'data/doc_level-sentiment/doc_level'

### Reading preprocess data

In [0]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [0]:
words_idx = read_pickle(data_path, 'words_idx.pkl')

In [0]:
idx_words = read_pickle(data_path, 'idx_words.pkl')

In [0]:
data = read_pickle(data_path, 'data.pkl')

In [0]:
label = read_pickle(data_path, 'label.pkl')

### Preparing training and validation set

In [11]:
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


In [0]:
rand_idx = np.arange(len(data))
np.random.shuffle(rand_idx)

data = data[rand_idx]
label = to_categorical(label)[rand_idx]

data_size = len(data)

test_x = data[0:1000]
test_y = label[0:1000]

dev_x = data[1000:5000]
dev_y = label[1000:5000]

train_x = data[5000:int(data_size)]
train_y = label[5000:int(data_size)]


In [0]:
maxlen = 300

import operator
words_idx = [x for (x, _) in sorted(words_idx.items(), key=operator.itemgetter(1))]

In [0]:
train_x_ = sequence.pad_sequences(train_x, maxlen)
dev_x_ = sequence.pad_sequences(dev_x, maxlen)
test_x_ = sequence.pad_sequences(test_x, maxlen)

In [0]:
train_x_ = np.array(train_x_)
train_y = np.array(train_y)

dev_x_ = np.array(dev_x_)
dev_y = np.array(dev_y)

test_x_ = np.array(test_x_)
test_y = np.array(test_y)

### Data iterator

In [0]:
class Dataiterator():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y

### LSTM Model for document level sentiment classification

In [0]:
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Input
from keras.models import Model

### Input Layer

In [0]:
### YOUR CODE HERE
sentence_input = Input(shape=(None,), dtype='int32', name='sentence_input')

### Layer to train embedding weights of words

In [19]:
### YOUR CODE HERE
vocab_size = len(words_idx)
word_emb = Embedding(vocab_size, 300, mask_zero=True, name='word_emb')
emb_output = word_emb(sentence_input)

Instructions for updating:
Colocations handled automatically by placer.


### RNN-based layer 

In [20]:
### YOUR CODE HERE
dropout= 0.5
recurrent_dropout = 0.1 
lstm_layer = LSTM(300, return_sequences=False, dropout=dropout, \
              recurrent_dropout=recurrent_dropout, name='lstm')(emb_output)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Prediction layer

In [0]:
### YOUR CODE HERE
densed = Dense(3, name='dense')(lstm_layer)
probs = Activation('softmax')(densed)

### Construct the model

In [0]:
### YOUR CODE HERE

model = Model(inputs=[sentence_input], outputs=probs)

In [0]:
import keras.optimizers as opt

In [0]:
optimizer = opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sentence_input (InputLayer)  (None, None)              0         
_________________________________________________________________
word_emb (Embedding)         (None, None, 300)         3000900   
_________________________________________________________________
lstm (LSTM)                  (None, 300)               721200    
_________________________________________________________________
dense (Dense)                (None, 3)                 903       
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 3,723,003
Trainable params: 3,723,003
Non-trainable params: 0
_________________________________________________________________


### Training with batch generator

In [0]:
batch_size = 32

In [0]:
train_steps_epoch = len(train_x_)/batch_size
batch_train_iter = Dataiterator(train_x_, train_y, batch_size)

In [0]:
val_steps_epoch = len(dev_x_)/batch_size
batch_val_iter = Dataiterator(dev_x_, dev_y, batch_size)

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_generator(model, batch_train_iter, batch_val_iter):
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]
    
    def train_gen():
        while True:
            train_batches = [[X, y] for X, y in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch
                
    def val_gen():
        while True:
            val_batches = [[X, y] for X, y in batch_val_iter]
            for val_batch in val_batches:
                yield val_batch
                
    history = model.fit_generator(train_gen(), validation_data=val_gen(), \
                                  validation_steps=val_steps_epoch, steps_per_epoch=train_steps_epoch, \
                                  epochs = 20, callbacks = earlystop_callbacks)
      

In [0]:
train_generator(model, batch_train_iter, batch_val_iter)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
782/781 [==============================] - 715s 914ms/step - loss: 0.9223 - categorical_accuracy: 0.5567 - val_loss: 0.8120 - val_categorical_accuracy: 0.6368
Epoch 2/20
782/781 [==============================] - 477s 609ms/step - loss: 0.7661 - categorical_accuracy: 0.6603 - val_loss: 0.7728 - val_categorical_accuracy: 0.6510
Epoch 3/20
782/781 [==============================] - 477s 610ms/step - loss: 0.6955 - categorical_accuracy: 0.7026 - val_loss: 0.7257 - val_categorical_accuracy: 0.6787
Epoch 4/20
782/781 [==============================] - 476s 609ms/step - loss: 0.6364 - categorical_accuracy: 0.7314 - val_loss: 0.7092 - val_categorical_accuracy: 0.6915
Epoch 5/20
782/781 [==============================] - 477s 610ms/step - loss: 0.5833 - categorical_accuracy: 0.7578 - val_loss: 0.7426 - val_categorical_accuracy: 0.6827
Epoch 6/20
782/781 [==============================] - 473s 605ms/step - loss: 0.5360 - categorical_acc